![](/Workspace/Users/infoblisstech@gmail.com/databricks-code-repo/6_lakeflow_pipelines/autoloader_file_ingestion_usecase1.png)

**Auto Loader** is Databricks’ cloud-native file ingestion engine for ingesting new files incrementally from object storage.

Supported Sources:
- AWS S3
- Azure ADLS Gen2
- Google Cloud Storage (GCS)

Modes:
- **Directory listing** - Directory listing scans storage paths to detect new files using the checkpoint feature
- File notification - Processes files as soon as they arrive at scale

####1. Directory listing
1. Spark lists directory
2. Detects new CSV or any files
3. Infers schema / evolves if needed
4. store the file into bronze layer
5. Updates checkpoint (file1 is processed...)
6. Waits for next job run to list directory and collect any new files into bronze

In [0]:
df1=spark.readStream.format("cloudFiles")\
.option("cloudFiles.format","csv")\
.option("cloudFiles.schemaEvolutionMode","addNewColumns")\
.option("cloudFiles.maxFilesPerTrigger",5)\
.option("cloudFiles.inferColumnTypes",True)\
.option("checkpointLocation", "/Volumes/catalog2_we47/schema2_we47/datalake/bronze/streamwrite1/_checkpoint")\
.option("cloudFiles.schemaLocation", "/Volumes/catalog2_we47/schema2_we47/datalake/bronze/streamwrite1/_schema")\
.option("header",True)\
.load(f"/Volumes/catalog2_we47/schema2_we47/datalake/sourcepath/")#this can be s3/adls/gcs
#.option("cloudFiles.useNotifications", "true") (Remove this option to enable directory listing)

In [0]:
df1.writeStream.trigger(availableNow=True)\
.option("mergeSchema", "true")\
.option("checkpointLocation", "/Volumes/catalog2_we47/schema2_we47/datalake/bronze/streamwrite1/_checkpoint")\
.option("cloudFiles.schemaLocation", "/Volumes/catalog2_we47/schema2_we47/datalake/bronze/streamwrite1/_schema")\
.start(f"/Volumes/catalog2_we47/schema2_we47/datalake/bronze/streamwrite1")

In [0]:
spark.read.format("delta").load("/Volumes/catalog2_we47/schema2_we47/datalake/bronze/streamwrite1/").orderBy("shipment_id").show(100)

Debugging in Databricks has been challenging for me in my current project, and I’m still learning the most effective approach. I’m trying to understand one of the existing pipelines, as it uses Structured Streaming, CDF. Currently, I’m adding show() at different stages in a feature branch to observe how the DataFrame transforms. Is this the right approach, or is there a better way to debug and understand the flow?